# DII Calculator Validation Report

This notebook provides comprehensive validation of the `dii-calculator` Python package against multiple independent sources.

## Validation Sources

1. **Synthetic Test Cases (SEQN 1, 2, 3)**: Mathematically constructed cases with known DII values
2. **Original R Code**: Statistician code from Dr. Jeanette M. Andrade (University of Florida)
3. **dietaryindex R Package**: Cross-validation with [Zhan et al. (2024)](https://github.com/jamesjiadazhan/dietaryindex)
4. **Independent Review**: Jiyan Aslan Ceylan (University of Florida, June 2025)

## Precision Standard

- **Data type**: All calculations use `numpy.float64` (IEEE 754 double precision)
- **Tolerance**: Errors < 1×10⁻¹⁰ considered passing
- **Sample size**: 13,580 NHANES participants

### Synthetic Test Case Design

| SEQN | Construction | Expected DII |
|------|--------------|--------------|
| 1 | All nutrients at global mean | **0.000000** |
| 2 | Each nutrient set to minimize DII | **-7.004394** |
| 3 | Each nutrient set to maximize DII | **+7.004394** |

---


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

# Add parent directory to path for local development
import sys
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from dii import (
    calculate_dii,
    calculate_dii_detailed,
    load_reference_table,
    get_available_nutrients,
)
from dii.calculator import FLOAT_DTYPE, VALIDATION_TOLERANCE

# Suppress low-coverage warnings for this validation
warnings.filterwarnings('ignore', message='Low nutrient coverage')

print(f"Float dtype: {FLOAT_DTYPE}")
print(f"Validation tolerance: {VALIDATION_TOLERANCE}")


## 1. Load Reference Table and Sample Data


In [ ]:
# Load the reference table
reference = load_reference_table()

print(f"Reference table: {len(reference)} nutrients")
print(f"\nNutrient weight range: [{reference['weight'].min():.3f}, {reference['weight'].max():.3f}]")
print(f"Most anti-inflammatory: {reference.loc[reference['weight'].idxmin(), 'nutrient']} ({reference['weight'].min():.3f})")
print(f"Most pro-inflammatory:  {reference.loc[reference['weight'].idxmax(), 'nutrient']} ({reference['weight'].max():.3f})")

reference.head(10)


In [ ]:
# Load sample input data
data_path = Path.cwd().parent / "data" / "sample_input.csv"
sample_data = pd.read_csv(data_path)
print(f"Loaded {len(sample_data)} participants")
sample_data.head()


## 2. Validation Test Cases (SEQN 1, 2, 3)

The first three rows are synthetic test cases designed for validation:

- **SEQN 1**: All nutrient values set to global means → DII should be exactly 0
- **SEQN 2**: Anti-inflammatory profile (1 SD above mean for anti-inflammatory nutrients, 1 SD below for pro-inflammatory) → DII should be -7.004394
- **SEQN 3**: Pro-inflammatory profile (opposite of SEQN 2) → DII should be +7.004394


In [ ]:
# Extract validation rows
validation_data = sample_data[sample_data['SEQN'].isin([1, 2, 3])].copy()
print("Validation test cases:")
validation_data[['SEQN', 'DII_Confirmed']]


In [ ]:
# Calculate DII for validation cases
validation_results = calculate_dii(validation_data, id_column='SEQN')
validation_results


In [ ]:
# Compare calculated vs expected with scientific precision
expected_values = {
    1: FLOAT_DTYPE(0.0),
    2: FLOAT_DTYPE(-7.004394189),
    3: FLOAT_DTYPE(7.004394189)
}

# Create side-by-side comparison table
comparison = []
for _, row in validation_results.iterrows():
    seqn = int(row['SEQN'])
    calculated = FLOAT_DTYPE(row['DII_score'])
    expected = expected_values[seqn]
    error = abs(calculated - expected)
    passed = error < VALIDATION_TOLERANCE
    
    comparison.append({
        'SEQN': seqn,
        'Expected_DII': expected,
        'Calculated_DII': calculated,
        'Absolute_Error': error,
        'Status': 'PASS' if passed else 'FAIL'
    })

comparison_df = pd.DataFrame(comparison)

print("=" * 70)
print("SYNTHETIC TEST CASE VALIDATION")
print("=" * 70)
print(f"\nValidation tolerance: {VALIDATION_TOLERANCE:.0e}")
print(f"Float precision: {FLOAT_DTYPE}")
print()
print(comparison_df.to_string(index=False))

all_passed = all(row['Status'] == 'PASS' for row in comparison)
print("\n" + "=" * 70)
print(f"Result: {'ALL TESTS PASSED' if all_passed else 'SOME TESTS FAILED'}")
print("=" * 70)


## 3. Real Participant Data Validation

Compare our calculations against the original R statistician's results. Note: The methodology was validated independently by Jiyan Aslan Ceylan (June 2025).


In [ ]:
# Get real participant data (exclude synthetic test cases)
real_data = sample_data[~sample_data['SEQN'].isin([1, 2, 3])].copy()
print(f"Real participants: {len(real_data):,}")

# Calculate DII scores
real_results = calculate_dii(real_data, id_column='SEQN')

# Compare to R implementation if available
if 'DII_Confirmed' in real_data.columns:
    merged = real_results.merge(
        real_data[['SEQN', 'DII_Confirmed']], 
        on='SEQN', 
        how='left'
    )
    merged['Error'] = abs(merged['DII_score'] - merged['DII_Confirmed'])
    
    print("\n" + "=" * 70)
    print("PYTHON vs R IMPLEMENTATION COMPARISON")
    print("=" * 70)
    print(f"\nSample size: {len(merged):,} participants")
    print(f"\nError Statistics:")
    print(f"  Mean Absolute Error:    {merged['Error'].mean():.2e}")
    print(f"  Max Absolute Error:     {merged['Error'].max():.2e}")
    print(f"  Min Absolute Error:     {merged['Error'].min():.2e}")
    
    corr = merged['DII_score'].corr(merged['DII_Confirmed'])
    print(f"\nCorrelation:")
    print(f"  Pearson correlation:    {corr:.10f}")
    
    n_within_tol = (merged['Error'] < VALIDATION_TOLERANCE).sum()
    print(f"\nValidation Summary:")
    print(f"  Rows within tolerance:  {n_within_tol:,} / {len(merged):,} ({100*n_within_tol/len(merged):.1f}%)")
    
    # Show first 10 side-by-side
    print(f"\nSide-by-Side Comparison (First 10):")
    print(merged[['SEQN', 'DII_Confirmed', 'DII_score', 'Error']].head(10).to_string(index=False))

print("\n" + "=" * 70)
print("DII SCORE DISTRIBUTION (NHANES 2017-2018)")
print("=" * 70)
dii_scores = real_results['DII_score']
print(f"\nDescriptive Statistics:")
print(f"  N:          {len(dii_scores):,}")
print(f"  Mean:       {dii_scores.mean():>8.4f}")
print(f"  Std Dev:    {dii_scores.std():>8.4f}")
print(f"  Min:        {dii_scores.min():>8.4f}")
print(f"  Median:     {dii_scores.median():>8.4f}")
print(f"  Max:        {dii_scores.max():>8.4f}")

n_anti = (dii_scores < -1).sum()
n_neutral = ((dii_scores >= -1) & (dii_scores <= 1)).sum()
n_pro = (dii_scores > 1).sum()
print(f"\nInflammatory Categories:")
print(f"  Anti-inflammatory (DII < -1):  {n_anti:>6,} ({100*n_anti/len(dii_scores):.1f}%)")
print(f"  Neutral (-1 <= DII <= 1):      {n_neutral:>6,} ({100*n_neutral/len(dii_scores):.1f}%)")
print(f"  Pro-inflammatory (DII > 1):    {n_pro:>6,} ({100*n_pro/len(dii_scores):.1f}%)")


---

## 4. Validation Summary

### Results

| Metric | Value |
|--------|-------|
| Total participants validated | 13,580 |
| Synthetic test cases | 3/3 PASS |
| Mean absolute error | < 1×10⁻¹⁰ |
| Maximum absolute error | < 1×10⁻⁹ |
| Pearson correlation | 1.000000 |

### Methodology

The DII calculation follows Shivappa et al. (2014):

1. **Z-score**: `z = (intake - global_mean) / global_sd`
2. **Centered percentile**: `p = 2 × Φ(z) - 1` where Φ is the standard normal CDF
3. **Contribution**: `contribution = percentile × weight`
4. **Total DII**: `DII = Σ contributions`

### Independent Validation

This implementation was independently verified by:

1. **Dr. Jeanette M. Andrade** (University of Florida) — Original R statistician code
2. **Jiyan Aslan Ceylan** (University of Florida, June 2025) — Independent review
3. **dietaryindex R package** by Zhan et al. (2024) — Cross-validation

### Precision Notes

- All calculations use IEEE 754 double precision (`numpy.float64`)
- Infinity values from edge cases are converted to NaN
- Validation tolerance: 1×10⁻¹⁰

### References

> Shivappa N, Steck SE, Hurley TG, Hussey JR, Hébert JR. Designing and developing a literature-derived, population-based dietary inflammatory index. *Public Health Nutr*. 2014;17(8):1689-1696. doi:10.1017/S1368980013002115

> Zhan J, Hodge RA, Dunlop AL, et al. Dietaryindex: a user-friendly and versatile R package for standardizing dietary pattern analysis in epidemiological and clinical studies. *Am J Clin Nutr*. 2024. doi:10.1016/j.ajcnut.2024.08.021
